### To-Do 
- [x] Remove columns with too many missing values
- [x] Use Pipelines
- MAE: 17609.825650684932
- Kaggle score: 16347.22769


In [1]:
# modules 
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib

In [2]:
#get the data
X_full = pd.read_csv('train.csv')
X_test_full = pd.read_csv('test.csv')

# remove rows with missing targets
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

# seperate target from predictors 
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# break off validation set from training set
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                               test_size=.2,
                                                               random_state=0)

In [3]:
# remove columns with lot of missing data
cols_missing = X_train_full.isnull().sum()
print(cols_missing[cols_missing > 0].sort_values(ascending=False))

PoolQC          1164
MiscFeature     1119
Alley           1097
Fence            954
FireplaceQu      551
LotFrontage      212
GarageYrBlt       58
GarageType        58
GarageFinish      58
GarageQual        58
GarageCond        58
BsmtFinType2      29
BsmtFinType1      28
BsmtExposure      28
BsmtCond          28
BsmtQual          28
MasVnrArea         6
MasVnrType         6
Electrical         1
dtype: int64


In [4]:
# remove if there is more than 300 values missing 
keep_cols_missing = cols_missing[cols_missing < 300]
print(keep_cols_missing[keep_cols_missing > 0].sort_values(ascending=False))

LotFrontage     212
GarageCond       58
GarageQual       58
GarageFinish     58
GarageYrBlt      58
GarageType       58
BsmtFinType2     29
BsmtFinType1     28
BsmtExposure     28
BsmtCond         28
BsmtQual         28
MasVnrArea        6
MasVnrType        6
Electrical        1
dtype: int64


In [5]:
keep_cols = [col for col in X_train_full.columns
            if X_train_full[col].isnull().sum() < 300]
print(keep_cols)

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', '

In [6]:
# keep columns with the least missing values
X_train = X_train_full[keep_cols].copy()
X_valid = X_valid_full[keep_cols].copy()
X_test = X_test_full[keep_cols].copy()

In [7]:
X_train.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
618  619          20       RL         90.0    11694   Pave      Reg   
870  871          20       RL         60.0     6600   Pave      Reg   
92    93          30       RL         80.0    13360   Pave      IR1   
817  818          20       RL          NaN    13265   Pave      IR1   
302  303          20       RL        118.0    13704   Pave      IR1   

    LandContour Utilities LotConfig  ... OpenPorchSF EnclosedPorch 3SsnPorch  \
618         Lvl    AllPub    Inside  ...         108             0         0   
870         Lvl    AllPub    Inside  ...           0             0         0   
92          HLS    AllPub    Inside  ...           0            44         0   
817         Lvl    AllPub   CulDSac  ...          59             0         0   
302         Lvl    AllPub    Corner  ...          81             0         0   

    ScreenPorch PoolArea MiscVal  MoSold  YrSold  SaleType  SaleCondition  
618         260        0       0       7    2007       New        Partial  
870           0        0       0       8    2009        WD         Normal  
92            0        0       0       8    2009        WD         Normal  
817           0        0       0       7    2008        WD         Normal  
302           0        0       0       1    2006        WD         Normal  

[5 rows x 75 columns]

In [8]:
# seperate numerical and categorical features
numerical_cols = [col for col in X_train.columns
                 if X_train[col].dtype in ['int64', 'float64']]

categorical_cols = [col for col in X_train.columns
                   if X_train[col].dtype == 'object']

In [9]:
# the pipeline 

# preprocessing numerical columns 
numerical_transformer = SimpleImputer(strategy='median')

# preprocessing categorical columns 
categorical_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bundl numerical and categorical preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

model = RandomForestRegressor(n_estimators=100, random_state=0)

# bundl preprocessor and model
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# fit the training data
my_pipeline.fit(X_train, y_train)

# get predictions
preds = my_pipeline.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17609.825650684932


In [10]:
# Save and generate submission
joblib.dump(my_pipeline, 'Models/rf_pipe_less_missing.pkl')

preds_test = my_pipeline.predict(X_test)

output = pd.DataFrame({
    'Id': X_test.index,
    'SalePrice': preds_test
})

output.to_csv('submission.csv', index=False)

the problem is we are using final_X_test.index as Id for our submission, but this id is 0 to n and this is not what is expected. It is expected to have id same with the sample submission.

In [12]:
submission = pd.read_csv('sample_submission.csv')
      
output = pd.DataFrame({
    'Id': submission.Id,
    'SalePrice': preds_test
})